In [9]:
import json
import boto3
from typing import Dict

SECRET_NAME = "ds4a-project-dev-db-connection-secret"


def get_secret(aws: boto3.Session, secret_name: str) -> Dict[str, str]:
    sm = aws.client("secretsmanager")
    response = sm.get_secret_value(SecretId=secret_name)
    secret = response["SecretString"]
    return json.loads(secret)


aws = boto3.Session()
params = get_secret(aws, SECRET_NAME)
params.keys()


dict_keys(['password', 'dbname', 'engine', 'port', 'dbInstanceIdentifier', 'host', 'username'])

In [10]:
import psycopg2


conn = psycopg2.connect(
    host=params["host"],
    database=params["dbname"],
    user=params["username"],
    password=params["password"],
)

cur = conn.cursor()
cur.execute("SELECT version()")
cur.fetchone()


('PostgreSQL 13.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-6), 64-bit',)

In [81]:
from psycopg2.extras import RealDictCursor

query = """--sql
    SELECT 
        m.name as name,
        m.code as code,
        mph.total as population,
        sum(sa.count) as suicide_attempts,
        CASE
            WHEN m.latitude = 'NaN' THEN NULL
            ELSE m.latitude
        END AS latitude,
        CASE
            WHEN m.longitude = 'NaN' THEN NULL
            ELSE m.longitude
        END AS longitude
    FROM suicideAttempts sa
    JOIN municipalities m 
        ON sa.municipality_id = m.code
    JOIN municipalityPopulationHistory mph 
        ON sa.municipality_id = mph.municipality_id AND sa.year = mph.year
    WHERE sa.year = 2016
    GROUP BY 
        name, code, population
"""

try:
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute(query)
    res = cursor.fetchall()
    conn.commit()
except Exception as e:
    conn.rollback()
    print(e)

resultset = [dict(row) for row in res]
print(f"{len(resultset)=:,d}")
resultset[:10]


len(resultset)=704


[{'name': 'Montecristo',
  'code': 13458,
  'population': 15415,
  'suicide_attempts': 2,
  'latitude': Decimal('8.29724472765'),
  'longitude': Decimal('-74.4714158637')},
 {'name': 'Usiacurí',
  'code': 8849,
  'population': 11713,
  'suicide_attempts': 3,
  'latitude': Decimal('10.742979811'),
  'longitude': Decimal('-74.9769840969')},
 {'name': 'Pinchote',
  'code': 68549,
  'population': 5062,
  'suicide_attempts': 1,
  'latitude': Decimal('6.53206093514'),
  'longitude': Decimal('-73.1729190191')},
 {'name': 'Malambo',
  'code': 8433,
  'population': 121589,
  'suicide_attempts': 30,
  'latitude': Decimal('10.856822692'),
  'longitude': Decimal('-74.7748637454')},
 {'name': 'Guasca',
  'code': 25322,
  'population': 14313,
  'suicide_attempts': 1,
  'latitude': Decimal('4.86671913331'),
  'longitude': Decimal('-73.8771423949')},
 {'name': 'Villamaría',
  'code': 17873,
  'population': 62261,
  'suicide_attempts': 36,
  'latitude': Decimal('5.043806686'),
  'longitude': Decimal('-

In [51]:
delete_statement = """--sql
DELETE FROM
    interfamilyViolence a
        USING interfamilyViolence b
WHERE
    a.id > b.id
    AND a.year = b.year
    AND a.count = b.count
    AND a.municipality_id = b.municipality_id;
"""

try:
    cur.execute(delete_statement)
    conn.commit()
except Exception as e:
    print(e)
    conn.rollback()


In [6]:
from psycopg2.extras import RealDictCursor

query = """--sql
SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'public'
"""

try:
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute(query)
    res = cursor.fetchall()
    conn.commit()
except:
    res = None
    conn.rollback()

res


[RealDictRow([('table_name', 'departments')]),
 RealDictRow([('table_name', 'departmentspopulationhistory')])]

In [8]:
conn.close()